In [100]:
import pandas as pd
import numpy as np
from csv import reader
from numpy import linalg as la
from sys import exit
from haversine import haversine
import math
from operator import itemgetter
from numpy import dot
from numpy.linalg import norm
#시각화 도구
import folium
from folium.features import DivIcon
import random
import matplotlib.pyplot as plt

# 전역 변수 설정
FILE_SAVE_DIR = './Data_set_0826/' #있다면 추가 해주기 (상대경로)
line_color_list = ['red', 'blue', 'green', 'purple', 'orange', 'darkred',
                    'lightred', 'beige', 'darkblue', 'darkgreen', 'cadetblue', 'darkpurple', 'white',
                    'pink', 'lightblue', 'lightgreen', 'gray', 'black', 'lightgray'] 
# Global Variable Setting

# Nicname = '5CAC8432AE4EC54B'
# Huzzi = '44754E5A077F32B4'
# oizi = '2DDB3706DE4F7B45'
kyounghwan = '3963650B38CAF112'
hyeryeong = '4CCF4C0B0F953026'
pyeongju = '8A57AE95DD4629A7'
joongho = '9E7F4CB4F4A6A6D4'
yujin = 'F9888CC2D3053565'
member_list = ['3963650B38CAF112','4CCF4C0B0F953026','8A57AE95DD4629A7','9E7F4CB4F4A6A6D4','F9888CC2D3053565']
PATIENT_UID = kyounghwan # 감염자 UID
TARGET_UID =  hyeryeong # 조사 대상자 UID

# Map Setting
Raw_map = folium.Map(location = [37.5505938572,127.074236903], zoom_start =100) # 세종대학교 중심 조사
Filter_map = folium.Map(location = [37.5505938572,127.074236903], zoom_start =100) # 세종대학교 중심 조사
mass_map = folium.Map(location = [37.5505938572,127.074236903], zoom_start =100) # 세종대학교 중심 조사
compare_map = folium.Map(location = [37.5505938572,127.074236903], zoom_start =100) # 세종대학교 중심 조사

#Call CSV file
def load_GPS_set(UID, scnario): # Data load
    try:
        if scnario == 1:
            data_set = pd.read_csv(FILE_SAVE_DIR + UID +"_gps"+".csv", sep=",", dtype='unicode')
            data_set['TIME STAMP'] = pd.to_datetime(data_set['loggingTime']).dt.strftime('%Y-%m-%d %H:%M:%s')
            is_scnario = data_set['scinario'] == '1'
            data_set = data_set[is_scnario]
            re_data_set = data_set[["TIME STAMP","uid","latitude","longitude","provider"]]
            return re_data_set
        elif scnario == 2:
            data_set = pd.read_csv(FILE_SAVE_DIR + UID +"_gps"+".csv", sep=",", dtype='unicode')
            data_set['TIME STAMP'] = pd.to_datetime(data_set['loggingTime']).dt.strftime('%Y-%m-%d %H:%M:%s')
            is_scnario = data_set['scinario'] == '2'
            data_set = data_set[is_scnario]
            re_data_set = data_set[["TIME STAMP","uid","latitude","longitude","provider"]]
            re_data_set.reset_index(inplace=True, drop=True)
            return re_data_set   
    except FileNotFoundError as e:
        pass

def load_ground_truth(UID, scnario):
    try:
        if scnario == 1:
            data_set = pd.read_csv(FILE_SAVE_DIR + UID +"_gps"+".csv", sep=",", dtype='unicode')
            data_set['TIME STAMP'] = pd.to_datetime(data_set['loggingTime']).dt.strftime('%Y-%m-%d %H:%M:%s')
            data_set.set_index("TIME STAMP", inplace = True)
            is_scnario = data_set['scinario'] == '1'
            data_set = data_set[is_scnario]
            re_data_set = data_set[["4CCF4C0B0F953026","3963650B38CAF112","8A57AE95DD4629A7","9E7F4CB4F4A6A6D4","F9888CC2D3053565"]]
            return re_data_set
        elif scnario == 2:
            data_set = pd.read_csv(FILE_SAVE_DIR + UID +"_gps"+".csv", sep=",", dtype='unicode')
            data_set['TIME STAMP'] = pd.to_datetime(data_set['loggingTime']).dt.strftime('%Y-%m-%d %H:%M:%s')
            data_set.set_index("TIME STAMP", inplace = True)
            is_scnario = data_set['scinario'] == '2'
            data_set = data_set[is_scnario]
            re_data_set = data_set[["4CCF4C0B0F953026","3963650B38CAF112","8A57AE95DD4629A7","9E7F4CB4F4A6A6D4","F9888CC2D3053565"]]
            return re_data_set
    
    except FileNotFoundError as e:
        pass
    
def load_exposed_ground_truth(scnario):
    try:
        data_set = pd.read_csv(FILE_SAVE_DIR + "exposed_gt.csv", sep=",", dtype='unicode')
        if scnario == 1:
            is_scnario = data_set['scinario'] == '1'
            data_set = data_set[is_scnario]
            re_data_set = data_set[["patient","4CCF4C0B0F953026","3963650B38CAF112","8A57AE95DD4629A7","9E7F4CB4F4A6A6D4","F9888CC2D3053565"]]
            return re_data_set
        elif scnario == 2:
            is_scnario = data_set['scinario'] == '2'
            data_set = data_set[is_scnario]
            re_data_set = data_set[["patient","4CCF4C0B0F953026","3963650B38CAF112","8A57AE95DD4629A7","9E7F4CB4F4A6A6D4","F9888CC2D3053565"]]
            return re_data_set
    
    except FileNotFoundError as e:
        pass

################### RAW DATA FUNC ######################
def extract_raw_data(UID, scnario):
    Dataframe = load_GPS_set(UID, scnario)
    marker_list = []
    for count in range(len(Dataframe.index)):
        marker_point_lati = float(Dataframe.loc[count,'latitude'])
        marker_point_long = float(Dataframe.loc[count,'longitude'])
        marker_point_time = Dataframe.loc[count,'TIME STAMP']
        marker_point = [marker_point_time, marker_point_lati, marker_point_long]
        marker_list.append(marker_point)
    return marker_list
    
################### ABOUT MAP DROWING FUNC #####################
def draw_Polyline_map(map, marker_list, draw_color):
    replace_list = []
    for marker_point in marker_list:
        replace_list.append(marker_point[1:3])
    folium.PolyLine(locations=replace_list,tooltip='Polyline',color=draw_color).add_to(map)

def draw_index_map(map, marker_list):
    count = len(marker_list)
    for marker_point in marker_list:
        folium.Marker(marker_point[1:3], icon=DivIcon(icon_size=(10,3),icon_anchor=(7,20),
        html='<div style="font-size: 10pt; color : black">'+str(count)+'</div>',
        )).add_to(map)
        count = count - 1

def draw_circle_map(map, marker_list,draw_radius,draw_color):
    for marker_point in marker_list:
        folium.CircleMarker(location=marker_point[1:3], radius=draw_radius, color= draw_color).add_to(map)
        
def draw_arrow_map(map, start_list, end_list, line_color, dot_color):
    for start_point in start_list:
        for end_point in end_list:
            if(start_point[0]==end_point[0]):
                try:
                    rot_x = end_point[1] - start_point[1]
                    rot_y = end_point[2] - start_point[2]
                    rot = math.acos(haversine([start_point[1],0],[end_point[1],0])/haversine(start_point[1:3],end_point[1:3]))
                    # folium.CircleMarker(location=end_point[1:3], radius=1, color= dot_color).add_to(map)
                    folium.PolyLine(locations=[start_point[1:3],end_point[1:3]],tooltip='Polyline',color=line_color).add_to(map)
                    folium.CircleMarker(location=start_point[1:3], radius=3, color= dot_color).add_to(map)
                    folium.RegularPolygonMarker(end_point[1:3], fill_color='blue', number_of_sides=3, radius=3, rotation=rot).add_to(map)
                except IndexError :
                    break
                except ZeroDivisionError:
                    break
                
def extractTime(UID, scnario):
    DataFrame = load_GPS_set(UID, scnario)
    timeStamp = np.unique(DataFrame["TIME STAMP"])
    return timeStamp

# kyounghwan_raw = extract_raw_data('3963650B38CAF112', 1)
# hyeryeong_raw = extract_raw_data('4CCF4C0B0F953026', 2)
# pyeongju_raw = extract_raw_data('8A57AE95DD4629A7', 1)
# joongho_raw = extract_raw_data('9E7F4CB4F4A6A6D4', 1)
# yujin_raw = extract_raw_data('F9888CC2D3053565', 1)


In [101]:
def moving_avg_filter(UID, scnario, filter_constant):
    marker_list = extract_raw_data(UID, scnario)
    avg_point_list = []
    for idx in range(len(marker_list)):
        x = marker_list[idx][1]
        y = marker_list[idx][2]
        avg_x = 0
        avg_y = 0
        for avgIdx in range(idx,idx + filter_constant):
            try:
                avg_x += float(marker_list[avgIdx][1])
                avg_y += float(marker_list[avgIdx][2])
            except IndexError:
                break
        avg_x = avg_x / filter_constant
        avg_y = avg_y / filter_constant
        point = [marker_list[idx][0], avg_x, avg_y]
        avg_point_list.append(point)
    return avg_point_list

# kyounghwan_avg = moving_avg_filter("3963650B38CAF112", 1, FILTER_CONSTANT)
# hyeryeong_avg = moving_avg_filter('4CCF4C0B0F953026', 1, FILTER_CONSTANT)
# pyeongju_avg = moving_avg_filter('8A57AE95DD4629A7', 1, FILTER_CONSTANT)
# joongho_avg = moving_avg_filter('9E7F4CB4F4A6A6D4', 1, FILTER_CONSTANT)
# yujin_avg = moving_avg_filter('F9888CC2D3053565', 1, FILTER_CONSTANT)

def moving_avg_filter_weighted(UID, scnario, weight):
    marker_list = extract_raw_data(UID, scnario)
    avg_point_list = []
    for idx in range(len(marker_list)):
        try:
            if len(avg_point_list) == 0:
                x = marker_list[idx][1]
                y = marker_list[idx][2]
            else:                
                x = avg_point_list[idx-1][1]
                y = avg_point_list[idx-1][2]
            x_fu = marker_list[idx+1][1]
            y_fu = marker_list[idx+1][2]
            avg_x = weight*x + (1-weight)*x_fu
            avg_y = weight*y + (1-weight)*y_fu
            point = [marker_list[idx][0], avg_x, avg_y]
            avg_point_list.append(point)
        except IndexError:
            break
    return avg_point_list


# kyounghwan_avg_weighted = moving_avg_filter_weighted("3963650B38CAF112", 1, WEIGHT)
# hyeryeong_avg_weighted = moving_avg_filter_weighted('4CCF4C0B0F953026', 1, WEIGHT)
# pyeongju_avg_weighted = moving_avg_filter_weighted('8A57AE95DD4629A7', 1, WEIGHT)
# joongho_avg_weighted = moving_avg_filter_weighted('9E7F4CB4F4A6A6D4', 1, WEIGHT)
# yujin_avg_weighted = moving_avg_filter_weighted('F9888CC2D3053565', 1, WEIGHT)


In [102]:
def trajectory_similarity(filter_menu, Patient_UID, Tester_UID, scnario, weight, filter_constant, DISTANCE_THRESHOLD):
    if filter_menu == "weighted":
        Patient_raw = moving_avg_filter_weighted(Patient_UID, scnario, weight)
        Tester_raw = moving_avg_filter_weighted(Tester_UID, scnario, weight)
    elif filter_menu == "simple" :
        Patient_raw = moving_avg_filter(Patient_UID, scnario,filter_constant)
        Tester_raw = moving_avg_filter(Tester_UID, scnario,filter_constant)
        
    time_stamp = extractTime(Patient_UID, scnario)
    contact_df = pd.DataFrame(columns=["TIME", "Distance", "tractory_sim"])
    contact_df["TIME"] = time_stamp
    contact_df.set_index("TIME", inplace = True)
    for time in time_stamp:
        distance = 0
        for p_idx in range(len(Patient_raw)):
            if Patient_raw[p_idx][0] == time:
                for t_idx in range(len(Tester_raw)):
                    if Tester_raw[t_idx][0] == time:
                        try:
                            distance = haversine(Patient_raw[p_idx][1:3],Tester_raw[t_idx][1:3], unit = 'm')
                            contact_df.loc[time,"Distance"] = distance
                            patient_speed_x = Patient_raw[p_idx + 1][1] - Patient_raw[p_idx][1]
                            patient_speed_y = Patient_raw[p_idx + 1][2] - Patient_raw[p_idx][2]
                            tester_speed_x = Tester_raw[t_idx + 1][1] - Tester_raw[t_idx][1]
                            tester_speed_y = Tester_raw[t_idx + 1][2] - Tester_raw[t_idx][2]
                            if distance <= DISTANCE_THRESHOLD:
                                cosine_sim = dot([patient_speed_x,patient_speed_y], [tester_speed_x,tester_speed_y]) / (norm([patient_speed_x,patient_speed_y]) * norm([tester_speed_x,tester_speed_y]))
                                contact_df.loc[time,"tractory_sim"] = cosine_sim
                            else : 
                                contact_df.loc[time,"tractory_sim"] = 0
                        except IndexError:
                            break
    return contact_df



def judgement_contact(result_df, patient_UID, tester_UID, scnario, OUTDOOR_SIM_THRESHOLD):
    gt_df = load_ground_truth(patient_UID, scnario)
    final_df = pd.DataFrame(columns=["TIME", "result", "groundtruth"])
    time_stamp = result_df.index
    final_df["TIME"] = time_stamp
    final_df.set_index("TIME", inplace = True)
    for time in time_stamp:
        try:
            if result_df.loc[time, "tractory_sim"] > OUTDOOR_SIM_THRESHOLD:
                final_df.loc[time, "result"] = 1
            else :
                final_df.loc[time, "result"] = 0
            final_df.loc[time, "groundtruth"] = gt_df.loc[time, tester_UID]
        except KeyError:
            pass
    tp = 0
    fp = 0
    fn = 0
    tn = 0
    for time in time_stamp:
        if final_df.loc[time, "result"] == 1 and final_df.loc[time, "groundtruth"] == '1':
            tp = tp + 1
        elif final_df.loc[time, "result"] == 1 and final_df.loc[time, "groundtruth"] == '0':
            fp = fp + 1
        elif final_df.loc[time, "result"] == 0 and final_df.loc[time, "groundtruth"] == '1':
            fn = fn + 1
        elif final_df.loc[time, "result"] == 0 and final_df.loc[time, "groundtruth"] == '0':
            tn = tn + 1
    return tp, fp, fn, tn, final_df

def add_avg_trajectory_similarity(result_df,t_contact, exposed_threshold):
    avg_list = []
    for time in result_df.index:
        avg_list.append(result_df.loc[time,"result"])
        if len(avg_list) == t_contact:
            if sum(avg_list) == exposed_threshold:
                return True
            else :
                avg_list.pop(0)
                continue
    return False

def judgement_exposed(result_df, patient_UID, tester_UID, scnario, t_contact, exposed_threshold, tp, fp, fn, tn):
    gt_df = load_exposed_ground_truth(scnario)
    exposed = add_avg_trajectory_similarity(result_df,t_contact, exposed_threshold)
    is_patient = gt_df["patient"] == patient_UID
    gt_df = gt_df[is_patient]
    gt_df.reset_index(inplace=True, drop=True)
    groundtruth = gt_df.loc[0,tester_UID]
   
    if exposed == True and groundtruth == '1':
        tp = tp + 1
    elif exposed == True and groundtruth == '0':
        fp = fp + 1
    elif exposed == False and groundtruth == '1':
        fn = fn + 1
    elif exposed == False and groundtruth == '0':
        tn = tn + 1

    return tp, fp, fn, tn


In [103]:
T_CONTACT = 10 #(min)
FILTER_CONSTANT = 3 #(min)
DISTANCE_THRESHOLD = 15 #(m)
WEIGHT = 0.5
OUTDOOR_SIM_THRESHOLD = 0.5
EXPOSED_THRESHOLD = 5


def all_result():
    e_tp = 0
    e_fn = 0
    e_fp = 0
    e_tn = 0
    TP = 0
    TN = 0
    FP = 0
    FN = 0
    for patient in member_list:
        patientUID = patient
        for tester in member_list:
            if tester != patient:
                testerUID = tester
                for scinario in [1,2]:
                    # r_TP, r_FP, r_FN, r_TN = judgement_contact(add_avg_trajectory_similarity(trajectory_similarity("weighted",patientUID,testerUID,scinario,WEIGHT,FILTER_CONSTANT)),patientUID,testerUID,scinario, TP, FP, FN, TN)
                    r_TP, r_FP, r_FN, r_TN, r_df = judgement_contact(trajectory_similarity("simple",patientUID,testerUID,scinario,WEIGHT,FILTER_CONSTANT,DISTANCE_THRESHOLD),patientUID,testerUID,scinario, OUTDOOR_SIM_THRESHOLD)
                    e_tp, e_fp, e_fn, e_tn = judgement_exposed(r_df,patientUID,testerUID,scinario,T_CONTACT,EXPOSED_THRESHOLD,e_tp, e_fp, e_fn, e_tn)
                    TP = TP + r_TP
                    TN = TN + r_TN
                    FP = FP + r_FP
                    FN = FN + r_FN
    precision = TP / (TP + FP)
    recall = TP / (TP + FN)
    accuracy = (TP+TN)/(TP+FN+FP+TN)
    f1_score = 2*((precision*recall)/(precision+recall))
    
    e_precision = e_tp / (e_tp + e_fp)
    e_recall = e_tp / (e_tp + e_fn)
    e_accuracy = (e_tp+e_tn)/(e_tp+e_fn+e_fp+e_tn)
    e_f1_score = 2*((e_precision*e_recall)/(e_precision+e_recall))
    
    print("============= contact ============= ")
    print("Accuracy : "+ str(accuracy))
    print("precision : "+ str(precision))
    print("recall : "+ str(recall))
    print("f1_score : "+ str(f1_score))
    
    print("============= exposed ============= ")
    print("Accuracy : "+ str(e_accuracy))
    print("precision : "+ str(e_precision))
    print("recall : "+ str(e_recall))
    print("f1_score : "+ str(e_f1_score))
    
all_result()

C:\Users\User\AppData\Local\Temp/ipykernel_26460/3042965075.py:27: RuntimeWarning: invalid value encountered in double_scalars
  cosine_sim = dot([patient_speed_x,patient_speed_y], [tester_speed_x,tester_speed_y]) / (norm([patient_speed_x,patient_speed_y]) * norm([tester_speed_x,tester_speed_y]))


============= result ============= 
Accuracy : 0.8081395348837209
precision : 0.8115942028985508
recall : 0.32
f1_score : 0.4590163934426229
4 0 8 28


In [104]:
draw_Polyline_map(Filter_map,hyeryeong_avg,"red")
draw_Polyline_map(Filter_map,pyeongju_avg,"blue")
draw_circle_map(Filter_map,hyeryeong_avg,3,"red")
draw_circle_map(Filter_map,pyeongju_avg,3,"blue")

NameError: name 'hyeryeong_avg' is not defined

In [ ]:
WEIGHT = 0.3
kyounghwan_avg_weighted = moving_avg_filter_weighted("3963650B38CAF112", 1, WEIGHT)
hyeryeong_avg_weighted = moving_avg_filter_weighted('4CCF4C0B0F953026', 1, WEIGHT)
pyeongju_avg_weighted = moving_avg_filter_weighted('8A57AE95DD4629A7', 1, WEIGHT)
joongho_avg_weighted = moving_avg_filter_weighted('9E7F4CB4F4A6A6D4', 1, WEIGHT)
yujin_avg_weighted = moving_avg_filter_weighted('F9888CC2D3053565', 1, WEIGHT)

draw_Polyline_map(mass_map,hyeryeong_avg_weighted,"red")
draw_Polyline_map(mass_map,pyeongju_avg_weighted,"blue")
draw_circle_map(mass_map,hyeryeong_avg_weighted,3,"red")
draw_circle_map(mass_map,pyeongju_avg_weighted,3,"blue")
# mass_map